In [224]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
from flask import Flask, request, render_template, session, redirect


In [225]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='2GB')
client

c:\Users\lebeq\anaconda3\envs\dev\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49374 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:49374/status,
Dashboard: http://127.0.0.1:49374/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49375,Workers: 1
Dashboard: http://127.0.0.1:49374/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:49382,Total threads: 4
Dashboard: http://127.0.0.1:49383/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:49378,


In [226]:
chicken_df = dd.read_csv('Resources/chicken_recipe.csv')
co2_df = dd.read_csv('Resources/co2.csv')


In [227]:
combined_df = dd.read_csv('Resources/*.csv')
combined_df


,recipe_id,recipe_name,cuisine_type,source,total_time,total_calories,calories_per_serving
npartitions=2,,,,,,,
,int64,object,object,object,float64,float64,float64
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [228]:
co2_df.head()

,recipe_id,total_co2,emission_class,diet_labels,total_weight,number_of_ingredients
0,0,4026.960000,F,"['High-Protein', 'Low-Carb', 'Low-Sodium']",408.000000,2
1,1,13435.534076,G,"['Low-Carb', 'Low-Sodium']",54.783084,2
2,2,6575.985636,F,"['Low-Fat', 'Low-Sodium']",879.622222,6
3,3,27074.433030,G,['Low-Carb'],2852.308290,6
4,4,13597.746146,F,['Low-Carb'],1409.710267,6


In [229]:
chicken_df.head()

,recipe_id,recipe_name,cuisine_type,source,total_time,total_calories,calories_per_serving
0,0,Poached Chicken for Tacos,['mexican'],Cookstr,120.0,432.480000,216.240000
1,1,Chicken Broth,['american'],Cookstr,240.0,117.308431,58.654216
2,2,Fig & Balsamic Chicken,['french'],Food52,60.0,1040.234000,346.744667
3,3,Basic Roast Chicken,['american'],Cookstr,120.0,3797.291855,632.881976
4,4,Ultimate Roast Chicken,['american'],My Recipes,0.0,2196.939412,366.156569


In [230]:
merged_df = dd.merge(chicken_df, co2_df, left_index=True, right_index=True)


In [231]:
merged_df.head(3)

,recipe_id_x,recipe_name,cuisine_type,source,total_time,total_calories,calories_per_serving,recipe_id_y,total_co2,emission_class,diet_labels,total_weight,number_of_ingredients
0,0,Poached Chicken for Tacos,['mexican'],Cookstr,120.0,432.480000,216.240000,0,4026.960000,F,"['High-Protein', 'Low-Carb', 'Low-Sodium']",408.000000,2
1,1,Chicken Broth,['american'],Cookstr,240.0,117.308431,58.654216,1,13435.534076,G,"['Low-Carb', 'Low-Sodium']",54.783084,2
2,2,Fig & Balsamic Chicken,['french'],Food52,60.0,1040.234000,346.744667,2,6575.985636,F,"['Low-Fat', 'Low-Sodium']",879.622222,6


In [232]:
cleaned_df = merged_df.drop(['recipe_id_x', 'recipe_id_y', 'source', 'emission_class', 'number_of_ingredients',], axis=1)
cleaned_df.head(3)

,recipe_name,cuisine_type,total_time,total_calories,calories_per_serving,total_co2,diet_labels,total_weight
0,Poached Chicken for Tacos,['mexican'],120.0,432.480000,216.240000,4026.960000,"['High-Protein', 'Low-Carb', 'Low-Sodium']",408.000000
1,Chicken Broth,['american'],240.0,117.308431,58.654216,13435.534076,"['Low-Carb', 'Low-Sodium']",54.783084
2,Fig & Balsamic Chicken,['french'],60.0,1040.234000,346.744667,6575.985636,"['Low-Fat', 'Low-Sodium']",879.622222


In [233]:
cleaned_df['Total_CO2_Per_Weight'] = (cleaned_df['total_co2']/cleaned_df['total_weight'])
cleaned_df.head()

,recipe_name,cuisine_type,total_time,total_calories,calories_per_serving,total_co2,diet_labels,total_weight,Total_CO2_Per_Weight
0,Poached Chicken for Tacos,['mexican'],120.0,432.480000,216.240000,4026.960000,"['High-Protein', 'Low-Carb', 'Low-Sodium']",408.000000,9.870000
1,Chicken Broth,['american'],240.0,117.308431,58.654216,13435.534076,"['Low-Carb', 'Low-Sodium']",54.783084,245.249683
2,Fig & Balsamic Chicken,['french'],60.0,1040.234000,346.744667,6575.985636,"['Low-Fat', 'Low-Sodium']",879.622222,7.475920
3,Basic Roast Chicken,['american'],120.0,3797.291855,632.881976,27074.433030,['Low-Carb'],2852.308290,9.492113
4,Ultimate Roast Chicken,['american'],0.0,2196.939412,366.156569,13597.746146,['Low-Carb'],1409.710267,9.645774


In [234]:
cleaned_df.groupby('recipe_name').total_co2.mean().compute()

recipe_name
Apricot Glazed Chicken                      5854.449735
Asian Grilled Chicken                      15042.841404
Balsamic Chicken                            4619.096871
Barbecue Chicken                           16800.357088
Barbecued Chicken                          37415.700620
                                               ...     
The Smith's Chicken-Fried Chicken Skins     7532.415217
Tuscan Chicken                             64975.389624
Twistin’ Chicken                            7741.530451
Ultimate Roast Chicken                     13597.746146
Un-Fried Chicken                           18357.268000
Name: total_co2, Length: 154, dtype: float64

In [235]:
cleaned_df.groupby('recipe_name').total_weight.mean().compute()

recipe_name
Apricot Glazed Chicken                      912.990463
Asian Grilled Chicken                      1562.425000
Balsamic Chicken                            499.867370
Barbecue Chicken                           1981.406628
Barbecued Chicken                          4649.321792
                                              ...     
The Smith's Chicken-Fried Chicken Skins    1455.729249
Tuscan Chicken                             2608.942891
Twistin’ Chicken                            938.947944
Ultimate Roast Chicken                     1409.710267
Un-Fried Chicken                           2551.908828
Name: total_weight, Length: 154, dtype: float64

In [236]:
cleaned_df.groupby('recipe_name').Total_CO2_Per_Weight.mean().compute()

recipe_name
Apricot Glazed Chicken                      6.412389
Asian Grilled Chicken                       9.627881
Balsamic Chicken                            9.240645
Barbecue Chicken                            8.479005
Barbecued Chicken                           8.047561
                                             ...    
The Smith's Chicken-Fried Chicken Skins     5.174324
Tuscan Chicken                             24.904872
Twistin’ Chicken                            8.244898
Ultimate Roast Chicken                      9.645774
Un-Fried Chicken                            7.193544
Name: Total_CO2_Per_Weight, Length: 154, dtype: float64

In [237]:
total_by_recipe_name = cleaned_df.groupby('recipe_name').recipe_name.count().compute()
total_by_recipe_name.head(25)

recipe_name
Apricot Glazed Chicken        1
Asian Grilled Chicken         1
Balsamic Chicken              1
Barbecue Chicken              1
Barbecued Chicken             1
Basic Roast Chicken           1
Belgian Chicken Waterzooi     1
Braised Chicken Gumbo         1
Braised Chicken Sausages      1
Buttermilk Brined Chicken     1
Candy Chicken                 1
Caramel Chicken               1
Cardamom Honey Chicken        1
Champagne-Can Chicken         1
Chicken "Stoup"               1
Chicken & chorizo traybake    1
Chicken & peppers             1
Chicken & sweetcorn ramen     1
Chicken Biryani               1
Chicken Broth                 2
Chicken Cacciatore            3
Chicken Carbonara             2
Chicken Chili                 1
Chicken Cordon Bleu           1
Chicken Couscous Salad        1
Name: recipe_name, dtype: int64

In [238]:
recipe_analysis = cleaned_df.groupby('recipe_name').agg({'total_co2': ['mean', 'std'], 'total_weight': ['mean', 'count']}).compute()
recipe_analysis

total_co2              \
                                                 mean         std   
recipe_name                                                         
Poached Chicken for Tacos                 4026.960000         NaN   
Chicken Broth                            20374.395391  9813.03178   
Fig & Balsamic Chicken                    6575.985636         NaN   
Basic Roast Chicken                      27074.433030         NaN   
Ultimate Roast Chicken                   13597.746146         NaN   
...                                               ...         ...   
The Smith's Chicken-Fried Chicken Skins   7532.415217         NaN   
Saucy chicken & vegetables                9824.902654         NaN   
Chicken Tortilla Soup                    12014.121638         NaN   
Chicken and Tortellini Soup              19460.936237         NaN   
Skillet Chicken and Rice                  9944.733747         NaN   

                                        total_weight        
                                                mean count  
recipe_name                                                 
Poached Chicken for Tacos                 408.000000     1  
Chicken Broth                              92.736182     2  
Fig & Balsamic Chicken                    879.622222     1  
Basic Roast Chicken                      2852.308290     1  
Ultimate Roast Chicken                   1409.710267     1  
...                                              ...   ...  
The Smith's Chicken-Fried Chicken Skins  1455.729249     1  
Saucy chicken & vegetables               1345.710341     1  
Chicken Tortilla Soup                    2231.996784     1  
Chicken and Tortellini Soup              1895.589533     1  
Skillet Chicken and Rice                 1442.532833     1  

[154 rows x 4 columns]

In [239]:
analysis_df = pd.DataFrame(recipe_analysis)
analysis_df

total_co2              \
                                                 mean         std   
recipe_name                                                         
Poached Chicken for Tacos                 4026.960000         NaN   
Chicken Broth                            20374.395391  9813.03178   
Fig & Balsamic Chicken                    6575.985636         NaN   
Basic Roast Chicken                      27074.433030         NaN   
Ultimate Roast Chicken                   13597.746146         NaN   
...                                               ...         ...   
The Smith's Chicken-Fried Chicken Skins   7532.415217         NaN   
Saucy chicken & vegetables                9824.902654         NaN   
Chicken Tortilla Soup                    12014.121638         NaN   
Chicken and Tortellini Soup              19460.936237         NaN   
Skillet Chicken and Rice                  9944.733747         NaN   

                                        total_weight        
                                                mean count  
recipe_name                                                 
Poached Chicken for Tacos                 408.000000     1  
Chicken Broth                              92.736182     2  
Fig & Balsamic Chicken                    879.622222     1  
Basic Roast Chicken                      2852.308290     1  
Ultimate Roast Chicken                   1409.710267     1  
...                                              ...   ...  
The Smith's Chicken-Fried Chicken Skins  1455.729249     1  
Saucy chicken & vegetables               1345.710341     1  
Chicken Tortilla Soup                    2231.996784     1  
Chicken and Tortellini Soup              1895.589533     1  
Skillet Chicken and Rice                 1442.532833     1  

[154 rows x 4 columns]

In [241]:
app = Flask(__name__)

@app.route('/', methods=("POST", "GET"))
def html_table():

    return render_template('simple.html',  tables=[analysis_df.to_html(classes='data')], titles = analysis_df.columns.values)



if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.13:5000
Press CTRL+C to quit
[2024-02-01 13:40:14,680] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "c:\Users\lebeq\anaconda3\envs\dev\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\lebeq\anaconda3\envs\dev\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\lebeq\anaconda3\envs\dev\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\lebeq\anaconda3\envs\dev\lib\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\lebeq\AppData\Local\Temp\ipykernel_27964\3186945728.py", line 6, in html_table
    return render_template('simple.html',  tables=[anal